## Encoder generator: 002

In [1]:
import os
import numpy


import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer


import sys
sys.path.append("..")

from wordsToNumbers import Corpus
from wordsToNumbers import fibonacci_vocabulary

from wordsToQubits import put_word_on_sphere

from utils import get_corpus_from_directory, working_window, get_word_from_sphere

from qencode.encoders import e2_classic
from qencode.training_circuits import swap_t
from qencode.qubits_arrangement import QubitsArrangement

In [2]:
np.random.seed(73)

## Corpus

In [3]:
corpus_path='/Users/voicutu/Desktop/Qountry/CountryMixt'

corpus_tex = get_corpus_from_directory(corpus_path, limit=1)

corpus= Corpus(corpus_tex)
print(corpus.prop())

nr. words:1648 
nr. distinct words: 499 
len.text/len.vocab:3.302605210420842


In [4]:
parameterize_vovabulary = fibonacci_vocabulary(corpus.vocabulary)

## Training set 

In [5]:
history_lenghth = 3

In [6]:
x,y = working_window(history_lenghth, splited_text=corpus.split_text)

In [7]:
print("len training set:", len(x))

len training set: 1645


## Working principles

## Just a simple encoder

### BIG encoder

In [8]:
shots = 2500
nr_trash=2
nr_latent=2

spec_big = QubitsArrangement(nr_trash, nr_latent, nr_swap=1, nr_ent=0)
print("Qubits:", spec_big.qubits)

#set up the device 
dev = qml.device("default.qubit", wires=spec_big.num_qubits)

Qubits: [0, 1, 2, 3, 4, 5, 6]


In [9]:
# circuit initializer
def circuit_initializer(words):
    for i in range(len(words)):
        put_word_on_sphere(words[i], qubit=i)


@qml.qnode(dev)
def encoder_B(init_params, encoder_params, reinit_state, spec):
    #initilaization
    circuit_initializer(init_params)
    setAux(spec, reinit_state)
    
    #encoder
    e2_classic(params, [*spec.latent_qubits, *spec.trash_qubits])

    #swap test 
    swap_t(spec)

    return [qml.probs(i) for i in spec.swap_qubits]

#### Training parameters

In [10]:
num_samples = len(x)
batch_size = 47

epochs = 500

learning_rate = 0.0003
beta1 = 0.9
beta2 = 0.999
opt = AdamOptimizer(learning_rate, beta1=beta1, beta2=beta2)

In [11]:
def fid_func(output):
    # Implemented as the Fidelity Loss
    # output[0] because we take the probability that the state after the 
    # SWAP test is ket(0), like the reference state
    fidelity_loss = 1 / output[0]
    return fidelity_loss

In [12]:
def cost(encoder_params, X):
    reinit_state = [0 for i in range(2 ** len(spec.aux_qubits))]
    reinit_state[0] = 1.0
    loss = 0.0
    for x in X:
        output = training_circuit_example(init_params=x[0], encoder_params=encoder_params, reinit_state=reinit_state)[0]
        f = fid_func(output)
        loss = loss + f
    return loss / len(X)

In [13]:
def fidelity(encoder_params, X):
    reinit_state = [0 for i in range(2 ** len(spec.aux_qubits))]
    reinit_state[0] = 1.0
    loss = 0.0
    for x in X:
        output = training_circuit_example(init_params=x[0], encoder_params=encoder_params, reinit_state=reinit_state)[0]
        f = output[0]
        loss = loss + f
    return loss / len(X)

In [14]:
def iterate_batches(X, batch_size):
    X1 = [torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits)))) for x in X]
    X2 = []
    for i in range(len(X1)):
        X2.append([X1[1], X[i][1]])
    X = X2
    random.shuffle(X)

    batch_list = []
    batch = []
    for x in X:
        if len(batch) < batch_size:
            batch.append(x)

        else:
            batch_list.append(batch)
            batch = []
    if len(batch) != 0:
        batch_list.append(batch)
    return batch_list

In [15]:
training_data = []

for i in range(num_samples):
    w_l=[ w for w in x[i]]
    w_l.append(y[i])
    training_data.append(w_l)
    
print("data example:",training_data[0])

data example: ['same', 'old', 'dive', 'same']


In [16]:
# initialize random encoder parameters
nr_encod_qubits = len(spec_big.trash_qubits) + len(spec_big.latent_qubits)
nr_par_encoder =  15 * int(nr_encod_qubits*(nr_encod_qubits-1)/2)
encoder_params = np.random.uniform(size=(1, nr_par_encoder), requires_grad=True)

reinit_state = [0 for i in range(2 ** len(spec_big.aux_qubits))]
reinit_state[0] = 1.0

print(qml.draw(encoder_B)(training_data[0], encoder_params, reinit_state, spec_big))

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed